 # Hidden Markov Models

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd


In [2]:
dice_data = np.load("rolls.npy")
print(dice_data)
np.random.seed(13)

[[2]
 [0]
 [3]
 ...
 [1]
 [3]
 [5]]


In [ ]:
split_index = len(dice_data) // 2
train_data = dice_data[:split_index]
test_data = dice_data[split_index:]
